In [ ]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape
from dash_extensions.javascript import assign

In [ ]:
# read_response checks if the response of the server is valid JSON

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Risposta non valida JSON!")
        print("Contenuto della risposta:", t.text)
        data = None 

# get_measurement_unit returns the measurement unit of a given pollutant

def get_measurement_unit(pollutant):
    t=requests.post(url="http://127.0.0.1:5000/api/units", json={"var_pollutant": pollutant}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data[0]

# get_province_shape returns a GeoDataFrame containing the geometries of the provinces
   
def get_province_shape():
    t=requests.get(url="http://127.0.0.1:5000/api/province_shape")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry_province'] = gdf['geometry_province'].apply(wkt.loads)
    gdf.set_geometry('geometry_province', inplace=True)
    return gdf

# list_sensors_and_pollutants gets the list of sensor and relative pollutants for a given station

def list_sensors_and_pollutants(list_stations):
    t=requests.post(url="http://127.0.0.1:5000/api/sensors_and_pollutants", json={"var_id_stazione": list_stations}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data

# get_station_location returns a GeoDataFrame containing the points of the stations

def get_station_location():
    t=requests.get(url="http://127.0.0.1:5000/api/station_location")
    data = read_response(t)
    gdf = gpd.GeoDataFrame(data)
    gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
    gdf.set_geometry('geometry', inplace=True)
    return gdf

############################################# FUNCTION TO POPULATE THE DROPDOWN #############################################

# pollutant_dropdown populates the dropdown with the pollutatnts that have at least one recorded value

def pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

# all_pollutant_dropdown populates the dropdown with all the pollutants, even those that have no recorded values

def all_pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/all_pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

############################################# FUNCTION FOR MAP VISUALIZATION #############################################

# df_to_dash_table converts a DataFrame to a Dash HTML table for display in the popup of EU_DV_1

def df_to_dash_table(df):
    return html.Table([
        html.Thead(html.Tr([html.Th(col) for col in df.columns])),
        html.Tbody([
            html.Tr([html.Td(df.iloc[i][col]) for col in df.columns]) for i in range(len(df))
        ])
    ], style={"maxWidth": "300px", "fontSize": "12px"})